/*
 * AutoEncoder for Sequence feature
 * Copyright (c) 2018-2019 Wei Zhou, ChaoRui Liu, All Rights Reserved.
 *
 * Permission is hereby granted, free of charge, to any person
 * obtaining a copy of this software and associated documentation
 * files (the "Software"), to deal in the Software without
 * restriction, including without limitation the rights to use,
 * copy, modify, merge, publish, distribute, sublicense, and/or sell
 * copies of the Software, and to permit persons to whom the
 * Software is furnished to do so, subject to the following
 * conditions:
 *
 * The above copyright notice and this permission notice shall be
 * included in all copies or substantial portions of the Software.
 *
 * THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
 * EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES
 * OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND
 * NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT
 * HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY,
 * WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
 * FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR
 * OTHER DEALINGS IN THE SOFTWARE.
 */

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from datetime import datetime
import math
import time
import h5py
import random

In [2]:
with h5py.File('localtion_seq_dataset.h5','r') as h5r:
    labeled_data = h5r['labeled'].value
    labeled_class = h5r['classes'].value
    labeled_data = labeled_data.reshape(len(labeled_data),20,4,1)
labeled_data=labeled_data.astype(np.float32)
labeled_class=labeled_class.astype(np.float32)
# labeled_data = tf.cast(labeled_data,tf.float32)
# labeled_class = tf.cast(labeled_class,tf.float32)
input_x = tf.placeholder(tf.float32, shape=(None, 20, 4, 1))
# input_x = tf.reshape(input_x,[-1,20,4,1])
print input_x.get_shape()

(?, 20, 4, 1)


In [3]:
def conv_op(input_op, name, kh, kw, n_out, dh, dw, p):  
    n_in = input_op.get_shape()[-1].value  
  
    with tf.name_scope(name) as scope:  
        kernel = tf.get_variable(scope + "w", 
                                 shape=[kh, kw, n_in, n_out],  
                                 dtype=tf.float32,  
                                 initializer=tf.contrib.layers.xavier_initializer_conv2d())  
        conv = tf.nn.conv2d(input_op, kernel, (1, dh, dw, 1), padding='VALID')  
        biases = tf.Variable(tf.constant(0.0, shape=[n_out], dtype=tf.float32), trainable=True, name='b')  
        z = tf.nn.bias_add(conv, biases)  
        activation = tf.nn.relu(z, name=scope)  
        p += [kernel, biases]  
        return activation  

In [4]:
def dconv_op(input_op, name, kh, kw, n_out, dh, dw, p, outputshape):  
    n_in = input_op.get_shape()[-1].value  
  
    with tf.name_scope(name) as scope:  
        kernel = tf.get_variable(scope + "w", 
                                 shape=[kh, kw, n_in, n_out],  
                                 dtype=tf.float32,  
                                 initializer=tf.contrib.layers.xavier_initializer_conv2d())  
        conv = tf.nn.conv2d_transpose(input_op, kernel,outputshape ,(1, dh, dw, 1), padding='VALID')  
        biases = tf.Variable(tf.constant(0.0, shape=[n_out], dtype=tf.float32), trainable=True, name='b')  
        z = tf.nn.bias_add(conv, biases)  
        activation = tf.nn.relu(z, name=scope)  
        p += [kernel, biases]  
        return activation

In [30]:
p = []
conv1_1 = conv_op(input_x, name="conv1_1", kh=4, kw=1, n_out=1, dh=1, dw=1, p=p)
print conv1_1.get_shape()
conv1_2 = conv_op(conv1_1, name="conv1_2", kh=6, kw=1, n_out=1, dh=1, dw=1, p=p)
print conv1_2.get_shape()

(?, 17, 4, 1)
(?, 12, 4, 1)


In [31]:
dconv2_1 = dconv_op(conv1_2, name="dconv2_1", kh=6, kw=1, n_out=1, dh=1, dw=1, p=p,outputshape=tf.shape(conv1_1))
print dconv2_1.get_shape()
dconv2_2 = dconv_op(dconv2_1, name="dconv2_2", kh=4, kw=1, n_out=1, dh=1, dw=1, p=p,outputshape=tf.shape(input_x))
print dconv2_2.get_shape()

(?, ?, ?, 1)
(?, ?, ?, 1)


In [33]:
x_pre = dconv2_2
x_ture = input_x
cost = tf.reduce_mean(tf.pow(x_pre - x_ture, 2))
optimizer = tf.train.RMSPropOptimizer(0.0002).minimize(cost)

In [34]:
data_index = []
for i in range(len(labeled_data)):
    data_index.append(i)
vild_index = random.sample(data_index,1000)
vild_seq, vild_class = labeled_data[vild_index[:]], labeled_class[vild_index[:]]

train_index = []
for i in range(len(data_index)):
    if data_index[i] in vild_index:
        if i%100==0:
            print i
    else:
        train_index.append(data_index[i])

1
2
8
11
17
18
24
29
35
36
37
39
44
50
53
56
65
67
70
72
83
85
87
88
94
102
111
112
115
117
124
134
135
138
146
149
167
172
174
175
184
187
200
203
212
215
218
219
224
225
226
228
234
242
245
246
254
255
267
273
285
308
310
317
321
327
329
348
349
357
364
369
371
376
384
388
389
390
392
403
404
407
409
416
420
428
433
434
436
442
448
454
455
469
475
487
490
491
500
501
509
514
517
532
538
542
545
556
576
577
579
582
598
599
602
603
613
624
628
630
639
643
653
657
659
664
666
668
671
680
681
685
692
697
702
708
711
713
729
732
737
750
751
755
769
779
780
786
790
792
793
800
805
808
816
820
821
826
827
833
834
835
849
852
877
885
892
895
900
904
908
911
920
921
922
924
926
943
956
982
993
996
997
1004
1012
1013
1016
1018
1020
1024
1030
1034
1045
1059
1066
1073
1077
1082
1083
1084
1089
1090
1098
1121
1122
1127
1129
1138
1142
1148
1151
1154
1159
1167
1168
1172
1187
1194
1219
1226
1234
1236
1242
1243
1254
1258
1260
1287
1288
1300
1301
1306
1310
1311
1330
1343
1344
1345
1350
1359
1366
1367
1

In [36]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    for i in range(1000):
        data_slice = random.sample(train_index,50)
        
        o,c = sess.run([optimizer,cost], feed_dict={input_x: labeled_data[data_slice[:]]})
        if i%100==0:
            print c

    print("Optimization Finished!")

0.249851
0.249847
0.24979
0.249747
0.249704
0.249655
0.249587
0.249507
0.249465
0.249389
0.249337
0.249249
0.249184
0.249137
0.249054
0.248957
0.248872
0.248812
0.24871
0.248653
0.248569
0.248505
0.248391
0.24831
0.248218
0.248122
0.24804
0.247946
0.247849
0.247761
0.24769
0.247587
0.247487
0.247416
0.247306
0.247217
0.247135
0.247048
0.246948
0.246851
0.246743
0.246667
0.246559
0.24647
0.246379
0.246299
0.246195
0.246096
0.245992
0.24589
0.245823
0.245714
0.245625
0.24552
0.245406
0.245334
0.245238
0.245153
0.245061
0.24497
0.244867
0.24476
0.244661
0.244581
0.244462
0.244369
0.244299
0.244196
0.244125
0.244009
0.243927
0.243831
0.243721
0.243615
0.243539
0.243462
0.243359
0.243265
0.243158
0.243069
0.242971
0.242888
0.242795
0.242698
0.242607
0.242496
0.242392
0.242324
0.242236
0.242143
0.242036
0.241952
0.241857
0.241766
0.241666
0.241562
0.241479
0.241393
0.241286
0.241221
0.241118
0.241028
0.240928
0.240836
0.240752
0.240655
0.24055
0.240473
0.240375
0.240281
0.24017
0.240093
0.24